In [172]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB

In [173]:
train_df = pd.read_csv('/kaggle/input/a-tour-of-machine-learning-2021/data_train.csv')
print("Missing Values", sum(train_df.isna().sum()))

In [174]:
y = train_df['Label']
x = train_df.drop(columns=['GeneId','Label'])
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

In [175]:
clf = RandomForestClassifier(n_estimators=500, max_depth = 10, max_features=10, random_state=42)
clf.fit(train_x,train_y)
importance = clf.feature_importances_
# summarize feature importance
lists = []
for i,v in enumerate(importance):
    if v > 0.01:
        lists.append(i)
print(lists)
train_x = train_x.iloc[:, lists]
test_x = test_x.iloc[:, lists]

In [179]:
clf1 = LogisticRegression(solver='liblinear', random_state=42)
clf2 = RandomForestClassifier(n_estimators=500, max_depth = 10, max_features=10, random_state=42)
clf3 = GaussianNB()
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1.fit(train_x, train_y)
eclf1.score(train_x, train_y)

In [158]:
# pipe = Pipeline(([('scaler', StandardScaler()), ('reg',LogisticRegression())]))
# pipe.fit(train_x, train_y)
# pipe.score(train_x, train_y)

In [180]:
eclf1.score(test_x, test_y)

In [181]:
y_pred = eclf1.predict(test_x)

In [182]:
confusion_matrix(test_y, y_pred)

In [183]:
test_df = pd.read_csv('/kaggle/input/a-tour-of-machine-learning-2021/data_test.csv')

In [184]:
test_df.head()

In [185]:
final = pd.DataFrame()
final['GeneId'] = test_df['GeneId']
test = test_df.drop(columns=['GeneId'])
# test = test.iloc[:,lists]

In [187]:
final['Label'] = eclf1.predict(test)

In [188]:
final.to_csv('output.csv', index=False)